# Convolutional Neural Networks: Step by Step


## Ejercicio

Vamos a implementar una capa de convolución (CONV) y pooling (POOL) usando numpy, inlcuimos forward propagation and backward propagation. 

**Notación**:
- El superindice $[l]$ denota un objeto de la $l^{th}$ capa. 
    - Ejemplo: $a^{[4]}$ representa la matriz de activacion de la $4^{th}$ capa. $W^{[5]}$ y $b^{[5]}$ son los parametros de la $5^{th}$ capa.


- El superindice $(i)$ denota un objeto del $i^{th}$ ejemplo. 
    - Ejemplo: $x^{(i)}$ es el $i^{th}$ ejemplo de entrenamiento de entrada.
    
    
- El supraindice $i$ denota la $i^{th}$ entrada de un vector.
    - Ejemplo: $a^{[l]}_i$ denota la $i^{th}$ de la capa de activación $l$, asumiendo que es una capa fully connected (FC).
    
    
- $n_H$, $n_W$ y $n_C$ denota respectivamente la altura (height), el ancho (width) y el numero de canales (channels) de una capa dada. Si queremos indicar los valores de la capa $l$, debemos escribir $n_H^{[l]}$, $n_W^{[l]}$, $n_C^{[l]}$. 
- $n_{H_{prev}}$, $n_{W_{prev}}$ y $n_{C_{prev}}$ denotan respectivamente la altura (height), el ancho (width) y el numero de canales (channels) de la capa anterior. Si queremos indicar los valores de la capa $l$, esto se escribe del siguiente modo $n_H^{[l-1]}$, $n_W^{[l-1]}$, $n_C^{[l-1]}$. 

## 1 - Paquetes

Primero importaremos todos los paquetes que son necesarios para la implementación de nuestra CNN. 
- [numpy](www.numpy.org).
- [matplotlib](http://matplotlib.org) es una biblioteca para mostrar graficas en Python.

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from NNUrudateana import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

## 2 - Esquema de trabajo

¡Implementaremos los componentes básicos de una red neuronal convolucional! Cada función que implementaremos tiene instrucciones detalladas que serviran como guía de lo que hay que hacer:

- Función de convolución, incluye:
    - Zero Padding
    - Convolve window 
    - Convolution forward
    - Convolution backward (opcional)
- Función de pooling, incluye:
    - Pooling forward
    - Create mask 
    - Distribute value
    - Pooling backward (opcional)
    
En este notebook implementaremos estas funciones desde cero utilizando `numpy`. En el siguiente notebook, usaremos las funciones de TensorFlow equivalentes para crear el siguiente modelo:

<img src="images/model.png" style="width:800px;height:300px;">

**Note** Para cada función hacia adelante existe su equivalente hacia atras, por lo tanto almacenaremos algunos parametros en cache para luego utilizarlos en el backprop. 

## 3 - Convolutional Neural Networks

Una capa de convolución transforma un volumen de entrada en un volumen de salida de diferente tamaño, como se muestra a continuación.

<img src="images/conv_nn.png" style="width:350px;height:200px;">

En esta parte vamos a construir cada paso de la capa convolucional. Pero primero vamos a implementar dos funciones auxiliares: una para el zero padding y la otra para calcular la función de convolución en si. 

### 3.1 - Zero-Padding

Zero-padding agrega ceros alrededor de los bordes de una imagen:

<img src="images/PAD.png" style="width:600px;height:400px;">
<caption><center> <u> <font color='purple'> **Figura 1** </u><font color='purple'>  : **Zero-Padding**<br> Imagen (3 canales, RGB) con padding de valor 2. </center></caption>

Los principales beneficios del padding son los siguientes:

- Nos permite usar una capa CONV sin necesariamente reducir la altura y el ancho de los volúmenes. Esto es importante para construir redes más profundas, ya que de lo contrario la altura/ancho se reduciría a medida que avanzamos a las capas más profundas. Un caso especial que es importante es el padding denominado "same", esto genera que la altura/ancho se conserva después de una capa de CONV, se hace el padding con cierto tamaño que genera que el tamaño de la entrada sea el mismo que el de la salida. 

- Nos ayuda a mantener más información sobre el borde de una imagen. Sin padding, muy pocos valores en la siguiente capa se verían afectados por los píxeles sobre los bordes de una imagen.

**Ejercicio**: Implementar la siguiente función, la cual realiza el padding de todas las imagenes del conjunto X con tamaño pad. [Usar np.pad](https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html). Nota: si queremos realizar el padding de la matriz "a" de dimensiones $(5,5,5,5,5)$ con `pad = 1` para la 2da dimensión, `pad = 3` para la 4ta dimensión y `pad = 0` para el resto, tenemos que hacer:
```python
a = np.pad(a, ((0,0), (1,1), (0,0), (3,3), (0,0)), mode='constant', constant_values = (0,0))
```

In [ ]:
# Función: zero_pad

def zero_pad(X, pad):
    """
    Pad con ceros todas las imagenes del conjunto X. El padding se aplica sobre la altura y el ancho de las imagenes, 
    esto se muestra en la Figura 1.
    
    Parametros:
    X -- python numpy array de dimensiones (m, n_H, n_W, n_C) representa un lote de m imagenes
    pad -- integer, cantidad de padding alrededor de cada imagen sobre las dimensiones de los ejes vertical y horizontal 
    
    Return:
    X_pad -- Conjunto de imagenes con padded de dimensiones (m, n_H + 2*pad, n_W + 2*pad, n_C)
    """
    
    ### START CODE HERE ### (≈ 1 line)
    X_pad = np.pad(X, ((0,0), (pad,pad), (pad,pad), (0,0)), mode='constant', constant_values = (0,0))
    ### END CODE HERE ###
    
    return X_pad

In [ ]:
np.random.seed(1)
x = np.random.randn(4, 3, 3, 2)
x_pad = zero_pad(x, 2)
print ("x.shape =\n", x.shape)
print ("x_pad.shape =\n", x_pad.shape)
print ("x[1,1] =\n", x[1,1])
print ("x_pad[1,1] =\n", x_pad[1,1])

x_res =  np.array([[ 0.90085595, -0.68372786], [-0.12289023, -0.93576943], [-0.26788808,  0.53035547]])

x_re_pad = np.array([[ 0., 0.], [ 0., 0.], [ 0., 0.], [ 0., 0.], [ 0., 0.], [ 0., 0.], [ 0., 0.]])

assert(x_pad.shape == (4, 7, 7, 2))
assert(np.isclose(x[1,1], x_res).all())
assert(np.isclose(x_pad[1,1], x_re_pad).all())

fig, axarr = plt.subplots(1, 2)
axarr[0].set_title('x')
axarr[0].imshow(x[0,:,:,0])
axarr[1].set_title('x_pad')
axarr[1].imshow(x_pad[0,:,:,0])

**Expected Output**:

```
x.shape =
 (4, 3, 3, 2)
x_pad.shape =
 (4, 7, 7, 2)
x[1,1] =
 [[ 0.90085595 -0.68372786]
 [-0.12289023 -0.93576943]
 [-0.26788808  0.53035547]]
x_pad[1,1] =
 [[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]
```

### 3.2 - Single step of convolution 

En esta parte, implementaremos un solo paso de convolución, en el que aplicamos el filtro a una sola posición de la imagen. Esto se usa para construir una unidad convolucional, que hace lo siguiente: 

- Toma un volumen de entrada 
- Applica un filtro a cada posición de la entrada
- Devuelve otro volumen (normalmente de diferente tamaño)

<img src="images/Convolution_schematic.gif" style="width:500px;height:300px;">
<caption><center> <u> <font color='purple'> **Figura 2** </u><font color='purple'>  : **Operación de convolución**<br> con un filtro de 3x3 y un stride de 1 (stride = cantidad que moveremos la ventana en cada slide) </center></caption>

En aplicaciones de computer vision, cada valor de la imagen de la izquierda corresponde a el valor de un pixel, realizar la convolución equivale a multiplicar esta ventana por los valores de los pixeles de la imagen y sumar todos estos valores más el bias. Implementaremos una funciona que realiza la convolución de una sola ventana, que se corresponde con la aplicación de un filtro sobre una posicion posible de la ventana el cual retorna un valor real para esta. 

Luego realizaremos este paso de convolución sobre todas las ventanas posible para completar una capa de convolucion

**Ejercicio**: Implementar conv_single_step(). [Pista](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.sum.html).


**Nota**: la variable b se pasara a la función como un numpy array.  Si añadimos un escalar (un float o integer) a un numpy array, el resultado sera un numpy array. En el caso particular que un numpy array contenga un unico valor, podemos castearlo a un float para convertirlo a un escalar.

In [ ]:
# FUNCTION: conv_single_step

def conv_single_step(a_slice_prev, W, b):
    """
    Aplicar un filtro definido con parametros W a un solo slide (a_slice_prev) que sera la salida   
    de la capa anterior.
    
    Parametros:
    a_slice_prev -- slice de los datos de entrada con dimensiones (f, f, n_C_prev)
    W -- Weight parametros de pesos de la ventana - matriz de dimensiones (f, f, n_C_prev)
    b -- Bias parametros de bias de la ventana - matriz de dimensiones (1, 1, 1)
    
    Return:
    Z -- un valor escalar, el resultado de la convolución de la ventana (W, b) sobre a_slice_prev de los datos de entrada
    """

    ### START CODE HERE ### (≈ 2 lines of code)
    # Esta variable representa el producto entre a_slice_prev y W. No agregar Bias.
    s = a_slice_prev * W
    # Sum sobre todas las entradas del volumen s.
    Z = np.sum(s, dtype=np.float)
    # Sumo el Bias b a Z. Considerar la NOTA para que Z sea un escalar.
    Z += float(b)
    ### END CODE HERE ###

    return Z

In [ ]:
import math
def truncate(number, digits) -> float:
    stepper = 10.0 ** digits
    return math.trunc(stepper * number) / stepper

np.random.seed(1)
a_slice_prev = np.random.randn(4, 4, 3)
W = np.random.randn(4, 4, 3)
b = np.random.randn(1, 1, 1)

Z_aux = -6.99908945
Z = conv_single_step(a_slice_prev, W, b)

print("Z =", truncate(Z, 8))

assert(truncate(Z, 8) == Z_aux)

**Expected Output**:
<table>
    <tr>
        <td>
            **Z**
        </td>
        <td>
            -6.99908945068
        </td>
    </tr>

</table>

### 3.3 - Convolutional Neural Networks - Forward pass

En el forward pass, tomaremos muchos filtros y realizaremos la convolución con el volumen de entrada. Cada 'convolución' nos dara una matriz de 2D de salida. Luego juntaremos estas salidas para obtener un volumen de 3D:

<center>
<video width="620" height="440" src="images/conv_kiank.mp4" type="video/mp4" controls>
</video>
</center>

**Ejercicio**: 
Implementar la funcion que realizara la convolución entre los filtros `W` sobre la capa de activación previa `A_prev`.
Esta función toma las siguientes entradas:
* `A_prev`, la salida de la capa anterior (considerado como un conjunto de m); 
* Weights lo denotamos como `W`.  El filtro con tamaño de ventana `f` por `f`.
* El vector de bias es `b`, donde cada filtro tiene su propio (solo un) bias. 

Además se agrega un diccionario de hyperparameters que contiene el valor del stride y del padding. 

**Pista**: 
1. Para seleccionar una slide de `2x2` en la esquina superior izquierda de la matriz "a_prev" (de dimension (5,5,3)), debemos hacer lo siguiente:
```python
a_slice_prev = a_prev[0:2,0:2,:]
```
Observar que esto genera un corte 3D que tiene altura 2, ancho 2 y profundidad 3. La profundidad es el número de canales.
Esto es util para definir `a_slice_prev`, usar los indices `start/end` que definiremos.
2. Para definir una slice necesitamos definir los vertices `vert_start`, `vert_end`, `horiz_start` y `horiz_end`. Esta figura puede sernos de ayuda para encontrar los vertices los cuales podemos definir usando h, w, f y s en el codigo.

<img src="images/vert_horiz_kiank.png" style="width:400px;height:300px;">
<caption><center> <u> <font color='purple'> **Figura 3** </u><font color='purple'>  : **Definición de una slide usando vertical and horizontal start/end (con un filtro de 2x2)** <br> Esta figura muestra un solo canal.  </center></caption>


**Formulas**:
Estas formulas sobre las dimensiones del resultado de una convolución a partir de las dimensiones de la entrada:
$$ n_H = \lfloor \frac{n_{H_{prev}} - f + 2 \times pad}{stride} \rfloor +1 $$
$$ n_W = \lfloor \frac{n_{W_{prev}} - f + 2 \times pad}{stride} \rfloor +1 $$
$$ n_C = \text{number of filters used in the convolution}$$

Para este ejericio, no nos preocuparemos por la implementación vectorizada, y vamos a implementar todo con for-loops ya que la idea base es lograr entender el funcionamiento de una CNN.

#### Consejos adicionales


* Deberian utilizar el corte de matrices (ej.`varname[0:1,:,3:5]`) para las siguientes variables:  
  `a_prev_pad` ,`W`, `b`  
  Copiar el código de la función y ejecutarlo fuera de la función definida, en celdas separadas. Para comparar que el subconjunto de cada matriz es del tamaño y la dimensión que espera.
* Para decidir cómo obtener vert_start, vert_end; horiz_start, horiz_end, recordar que estos son índices de la capa anterior.  
  Los índices de la capa de salida se denotan con `h` y` w`.  
* Asegúrarse de que `a_slice_prev` tenga altura, ancho y profundidad.
* Recordar que `a_prev_pad` es un subconjunto de `A_prev_pad`.  
  Pensar cuál debería usarse dentro de los bucles for.

In [ ]:
# FUNCTION: conv_forward

def relu(X):
    return np.maximum(0,X)

def conv_forward(A_prev, W, b, hparameters):
    """
    Implementar el forward propagation para una capa de convolución
    
    Parametros:
    A_prev -- salida de activación de la capa anterior, 
        numpy array de dimensión (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array de dimensión (f, f, n_C_prev, n_C)
    b -- Biases, numpy array de dimensión (1, 1, 1, n_C)
    hparameters -- diccionario de python que contiene los valores de "stride" y "pad"
        
    Returns:
    Z -- salida de la capa de convolución, numpy array de dimensiones (m, n_H, n_W, n_C)
    cache -- cache de los valores necesarios para la función de conv_backward()
    """
    
    ### START CODE HERE ###
    # Obtener dimensiones de A_prev's (≈1 linea)  
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    # Obtener dimensiones de W's (≈1 linea)
    (f, f, n_C_prev, n_C) = W.shape
    
    # Obtener informacion de los "hparameters" (≈2 lineas)
    stride = hparameters["stride"]
    pad = hparameters["pad"]
    
    # Calcular las dimensiones de la salida de la capa de CONV usando las formulas vistas arriba. 
    # Pista: usar int() para aplicar el 'floor' de la función. (≈2 lineas)
    n_H = int((n_H_prev - f + 2*pad) / stride) + 1
    n_W = int((n_W_prev - f + 2*pad) / stride) + 1
    
    # Inicializar el volumen de salida Z con zeros. (≈1 linea)
    Z = np.zeros((m, n_H, n_W, n_C))
    
    # Crear A_prev_pad con el padding aplicado a A_prev
    A_prev_pad = zero_pad(A_prev, pad)
    
    for i in range(0, m):                   # loop sobre el conjunto de ejemplos de entrenamiento
        a_prev_pad = A_prev_pad[i] # Seleccionar el i-esimo ejemplo de entrenamiento con el padding aplicado
        for h in range(0, n_H):             # loop sobre el eje vertical del volumen de salida
            # Inicializar la arista vertical del correspondiente "slice" (≈2 lineas)
            vert_start = h * stride
            vert_end = vert_start + f
            
            for w in range(0, n_W):       # loop sobre el eje horizontal del volumen de salida
                # Inicializar la arista horizontal del correspondiente "slice" (≈2 lineas)
                horiz_start = w * stride
                horiz_end = horiz_start + f
                
                for c in range(0, n_C):   # loop sobre los canales (= #filters) del volumen de salida
                                        
                    # Usar la esquina para definir el slice (3D) de a_prev_pad. (≈1 linea)
                    a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]
                    
                    # Convolucionar la slice (3D) con el filtro correcto W y bias b, para obtener una neurona de salida. (≈3 lineas)
                    weights = W[:,:,:,c]
                    biases = b[:,:,:,c]
                    Z[i, h, w, c] = relu(conv_single_step(a_slice_prev, weights, biases))
                                        
    ### END CODE HERE ###
    
    assert(Z.shape == (m, n_H, n_W, n_C))
    
    # Guardamos la información en "cache" para el backprop
    cache = (A_prev, W, b, hparameters)
    
    return Z, cache

Finalmente, la capa CONV también debe contener una activación, en cuyo caso agregaríamos la siguiente línea de código:

```python
# Aplicar la función de activación
A[i, h, w, c] = activation(Z[i, h, w, c])
```

Esto no es necesario en este ejercicio.


## 4 - Pooling layer 

La capa de agrupación (POOL) reduce la altura y el ancho de la entrada. Ayuda a reducir el computo necesario, así como a hacer que los detectores de características sean más invariantes a la posición en la entrada. Los dos tipos de capas de agrupación son:

- Max-pooling layer: en las slides de entrada aplicamos ventanas de dimensiones ($f, f$) y almacenamos el maximo valor de la ventana en la salida.

- Average-pooling layer: en las slides de entrada aplicamos ventanas de dimensiones ($f, f$) y almacenamos el valor promedio de la ventana en la salida.

<table>
<td>
<img src="images/max_pool1.png" style="width:500px;height:300px;">
<td>

<td>
<img src="images/a_pool.png" style="width:500px;height:300px;">
<td>
</table>

Estas capas de agrupación no tienen parámetros para que la retropropagación entrene. Sin embargo, tienen hiperparámetros como el tamaño de la ventana $f$. Esto especifica la altura y el ancho de la ventana $f \times f$ en la que calcularía el *maximo* o *promedio*.

### 4.1 - Forward Pooling
Ahora, vamos a implementar MAX-POOL y AVG-POOL, en la misma función. 

**Ejercicio**: Implementar la forward pass sobre una pooling layer.

**Recordatorio**:
Las formulas sobre la dimensión del volumen de salida en relación al volumen de entrada son:

$$ n_H = \lfloor \frac{n_{H_{prev}} - f}{stride} \rfloor +1 $$

$$ n_W = \lfloor \frac{n_{W_{prev}} - f}{stride} \rfloor +1 $$

$$ n_C = n_{C_{prev}}$$

In [ ]:
# FUNCTION: pool_forward

def pool_forward(A_prev, hparameters, mode = "max"):
    """
    Implementar la forward pass de la pooling layer
    
    Parametros:
    A_prev -- Datos de entrada, numpy array de dimension (m, n_H_prev, n_W_prev, n_C_prev)
    hparameters -- diccionario de python contiene los valores de "f" y "stride"
    mode -- el pooling mode que te gustaria usar, definido como un string que puede tomar estos valores ("max" or "average")
    
    Returns:
    A -- salida de la pool layer, numpy array de dimensión (m, n_H, n_W, n_C)
    cache -- cache usado en el backward pass de la pooling layer, contiene la entrada y los hparameters 
    """
    
    # Obtener las dimensiones del volumen de entrada
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Obtener los hyperparameters de "hparameters"
    f = hparameters["f"]
    stride = hparameters["stride"]
    
    # Definir las dimensiones de la salida
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev
    
    # Inicializar la matriz de salida A
    A = np.zeros((m, n_H, n_W, n_C))              
    
    ### START CODE HERE ###
    for i in range(0, m):                         # loop sobre los ejemplos de entrenamiento
        a_prev = A_prev[i]
        for h in range(0, n_H):                     # loop sobre el eje vertical del volumen de salida
            # Calcular la arista vertical de la "slice" (≈2 lineas)
            vert_start = h * stride
            vert_end = vert_start + f
            
            for w in range(0, n_W):                 # loop sobre el eje horizontal del volumen de salida
                # Calcular la arista horizontal de la "slice" (≈2 lines)
                horiz_start = w * stride
                horiz_end = horiz_start + f
                
                for c in range (n_C):            # loop sobre los canales del volumen de salida
                    
                    # Usar la esquina definida por el slice en el i-esimo ejemplo de entrenamiento A_prev, del canal c. (≈1 linea)
                    a_prev_slice = a_prev[vert_start:vert_end, horiz_start:horiz_end, c]
                    
                    # Calcular el pooling sobre el slice. 
                    # Usamos un if para diferenciar el modo. 
                    # Calcular el maximo y la media del NUMPY ARRAY.
                    if mode == "max":
                        A[i, h, w, c] = np.max(a_prev_slice)
                    elif mode == "average":
                        A[i, h, w, c] = np.mean(a_prev_slice)
    
    ### END CODE HERE ###
    
    # Guardamos la entrada y los hparameters en el "cache" para el pool_backward()
    cache = (A_prev, hparameters)
    
    assert(A.shape == (m, n_H, n_W, n_C))
    
    return A, cache

Congratulations! You have now implemented the forward passes of all the layers of a convolutional network. 

The remainder of this notebook is optional, and will not be graded.


## 5 - Backpropagation in convolutional neural networks (OPTIONAL / UNGRADED)

In modern deep learning frameworks, you only have to implement the forward pass, and the framework takes care of the backward pass, so most deep learning engineers don't need to bother with the details of the backward pass. The backward pass for convolutional networks is complicated. If you wish, you can work through this optional portion of the notebook to get a sense of what backprop in a convolutional network looks like. 

When in an earlier course you implemented a simple (fully connected) neural network, you used backpropagation to compute the derivatives with respect to the cost to update the parameters. Similarly, in convolutional neural networks you can calculate the derivatives with respect to the cost in order to update the parameters. The backprop equations are not trivial and we did not derive them in lecture, but we will briefly present them below.

### 5.1 - Convolutional layer backward pass 

Let's start by implementing the backward pass for a CONV layer. 

#### 5.1.1 - Computing dA:
This is the formula for computing $dA$ with respect to the cost for a certain filter $W_c$ and a given training example:

$$ dA += \sum _{h=0} ^{n_H} \sum_{w=0} ^{n_W} W_c \times dZ_{hw} \tag{1}$$

Where $W_c$ is a filter and $dZ_{hw}$ is a scalar corresponding to the gradient of the cost with respect to the output of the conv layer Z at the hth row and wth column (corresponding to the dot product taken at the ith stride left and jth stride down). Note that at each time, we multiply the the same filter $W_c$ by a different dZ when updating dA. We do so mainly because when computing the forward propagation, each filter is dotted and summed by a different a_slice. Therefore when computing the backprop for dA, we are just adding the gradients of all the a_slices. 

In code, inside the appropriate for-loops, this formula translates into:
```python
da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
```

#### 5.1.2 - Computing dW:
This is the formula for computing $dW_c$ ($dW_c$ is the derivative of one filter) with respect to the loss:

$$ dW_c  += \sum _{h=0} ^{n_H} \sum_{w=0} ^ {n_W} a_{slice} \times dZ_{hw}  \tag{2}$$

Where $a_{slice}$ corresponds to the slice which was used to generate the activation $Z_{ij}$. Hence, this ends up giving us the gradient for $W$ with respect to that slice. Since it is the same $W$, we will just add up all such gradients to get $dW$. 

In code, inside the appropriate for-loops, this formula translates into:
```python
dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
```

#### 5.1.3 - Computing db:

This is the formula for computing $db$ with respect to the cost for a certain filter $W_c$:

$$ db = \sum_h \sum_w dZ_{hw} \tag{3}$$

As you have previously seen in basic neural networks, db is computed by summing $dZ$. In this case, you are just summing over all the gradients of the conv output (Z) with respect to the cost. 

In code, inside the appropriate for-loops, this formula translates into:
```python
db[:,:,:,c] += dZ[i, h, w, c]
```

**Exercise**: Implement the `conv_backward` function below. You should sum over all the training examples, filters, heights, and widths. You should then compute the derivatives using formulas 1, 2 and 3 above. 

In [ ]:
def conv_backward(dZ, cache):
    """
    Implement the backward propagation for a convolution function
    
    Arguments:
    dZ -- gradient of the cost with respect to the output of the conv layer (Z), numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward(), output of conv_forward()
    
    Returns:
    dA_prev -- gradient of the cost with respect to the input of the conv layer (A_prev),
               numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    dW -- gradient of the cost with respect to the weights of the conv layer (W)
          numpy array of shape (f, f, n_C_prev, n_C)
    db -- gradient of the cost with respect to the biases of the conv layer (b)
          numpy array of shape (1, 1, 1, n_C)
    """
    
    ### START CODE HERE ###
    # Retrieve information from "cache"
    (A_prev, W, b, hparameters) = cache
    
    # Retrieve dimensions from A_prev's shape
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Retrieve dimensions from W's shape
    (f, f, n_C_prev, n_C) = W.shape
    
    # Retrieve information from "hparameters"
    stride = hparameters["stride"]
    pad = hparameters["pad"]
    
    # Retrieve dimensions from dZ's shape
    (m, n_H, n_W, n_C) = dZ.shape
    
    # Initialize dA_prev, dW, db with the correct shapes
    dA_prev = np.zeros(A_prev.shape)                           
    dW = np.zeros(W.shape)   
    db = np.zeros(b.shape)   

    # Pad A_prev and dA_prev
    A_prev_pad = zero_pad(A_prev, pad)
    dA_prev_pad = zero_pad(dA_prev, pad)
    
    for i in range(m):                       # loop over the training examples
        
        # select ith training example from A_prev_pad and dA_prev_pad
        a_prev_pad = A_prev_pad[i]
        da_prev_pad = dA_prev_pad[i]
        
        for h in range(n_H):                   # loop over vertical axis of the output volume
            for w in range(n_W):               # loop over horizontal axis of the output volume
                for c in range(n_C):           # loop over the channels of the output volume
                    
                    # Find the corners of the current "slice"
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    
                    # Use the corners to define the slice from a_prev_pad
                    a_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

                    # Update gradients for the window and the filter's parameters using the code formulas given above
                    da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
                    dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
                    db[:,:,:,c] += dZ[i, h, w, c]
                    
        # Set the ith training example's dA_prev to the unpadded da_prev_pad (Hint: use X[pad:-pad, pad:-pad, :])
        dA_prev[i, :, :, :] = dA_prev_pad[i, pad:-pad, pad:-pad, :]
    ### END CODE HERE ###
    
    # Making sure your output shape is correct
    assert(dA_prev.shape == (m, n_H_prev, n_W_prev, n_C_prev))
    
    return dA_prev, dW, db

## 5.2 Pooling layer - backward pass

Next, let's implement the backward pass for the pooling layer, starting with the MAX-POOL layer. Even though a pooling layer has no parameters for backprop to update, you still need to backpropagation the gradient through the pooling layer in order to compute gradients for layers that came before the pooling layer. 

### 5.2.1 Max pooling - backward pass  

Before jumping into the backpropagation of the pooling layer, you are going to build a helper function called `create_mask_from_window()` which does the following: 

$$ X = \begin{bmatrix}
1 && 3 \\
4 && 2
\end{bmatrix} \quad \rightarrow  \quad M =\begin{bmatrix}
0 && 0 \\
1 && 0
\end{bmatrix}\tag{4}$$

As you can see, this function creates a "mask" matrix which keeps track of where the maximum of the matrix is. True (1) indicates the position of the maximum in X, the other entries are False (0). You'll see later that the backward pass for average pooling will be similar to this but using a different mask.  

**Exercise**: Implement `create_mask_from_window()`. This function will be helpful for pooling backward. 
Hints:
- [np.max()]() may be helpful. It computes the maximum of an array.
- If you have a matrix X and a scalar x: `A = (X == x)` will return a matrix A of the same size as X such that:
```
A[i,j] = True if X[i,j] = x
A[i,j] = False if X[i,j] != x
```
- Here, you don't need to consider cases where there are several maxima in a matrix.

In [ ]:
def create_mask_from_window(x):
    """
    Creates a mask from an input matrix x, to identify the max entry of x.
    
    Arguments:
    x -- Array of shape (f, f)
    
    Returns:
    mask -- Array of the same shape as window, contains a True at the position corresponding to the max entry of x.
    """
    
    ### START CODE HERE ### (≈1 line)
    mask = (x == np.max(x))
    ### END CODE HERE ###
    
    return mask

Why do we keep track of the position of the max? It's because this is the input value that ultimately influenced the output, and therefore the cost. Backprop is computing gradients with respect to the cost, so anything that influences the ultimate cost should have a non-zero gradient. So, backprop will "propagate" the gradient back to this particular input value that had influenced the cost. 

### 5.2.2 - Average pooling - backward pass 

In max pooling, for each input window, all the "influence" on the output came from a single input value--the max. In average pooling, every element of the input window has equal influence on the output. So to implement backprop, you will now implement a helper function that reflects this.

For example if we did average pooling in the forward pass using a 2x2 filter, then the mask you'll use for the backward pass will look like: 
$$ dZ = 1 \quad \rightarrow  \quad dZ =\begin{bmatrix}
1/4 && 1/4 \\
1/4 && 1/4
\end{bmatrix}\tag{5}$$

This implies that each position in the $dZ$ matrix contributes equally to output because in the forward pass, we took an average. 

**Exercise**: Implement the function below to equally distribute a value dz through a matrix of dimension shape. [Hint](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ones.html)

In [ ]:
def distribute_value(dz, shape):
    """
    Distributes the input value in the matrix of dimension shape
    
    Arguments:
    dz -- input scalar
    shape -- the shape (n_H, n_W) of the output matrix for which we want to distribute the value of dz
    
    Returns:
    a -- Array of size (n_H, n_W) for which we distributed the value of dz
    """
    
    ### START CODE HERE ###
    # Retrieve dimensions from shape (≈1 line)
    (n_H, n_W) = shape
    
    # Compute the value to distribute on the matrix (≈1 line)
    average = dz/(n_H + n_W)
    
    # Create a matrix where every entry is the "average" value (≈1 line)
    a = np.ones(shape) * average
    ### END CODE HERE ###
    
    return a

### 5.2.3 Putting it together: Pooling backward 

You now have everything you need to compute backward propagation on a pooling layer.

**Exercise**: Implement the `pool_backward` function in both modes (`"max"` and `"average"`). You will once again use 4 for-loops (iterating over training examples, height, width, and channels). You should use an `if/elif` statement to see if the mode is equal to `'max'` or `'average'`. If it is equal to 'average' you should use the `distribute_value()` function you implemented above to create a matrix of the same shape as `a_slice`. Otherwise, the mode is equal to '`max`', and you will create a mask with `create_mask_from_window()` and multiply it by the corresponding value of dA.

In [ ]:
def pool_backward(dA, cache, mode = "max"):
    """
    Implements the backward pass of the pooling layer
    
    Arguments:
    dA -- gradient of cost with respect to the output of the pooling layer, same shape as A
    cache -- cache output from the forward pass of the pooling layer, contains the layer's input and hparameters 
    mode -- the pooling mode you would like to use, defined as a string ("max" or "average")
    
    Returns:
    dA_prev -- gradient of cost with respect to the input of the pooling layer, same shape as A_prev
    """
    
    ### START CODE HERE ###
    
    # Retrieve information from cache (≈1 line)
    (A_prev, hparameters) = cache
    
    # Retrieve hyperparameters from "hparameters" (≈2 lines)
    stride = hparameters["stride"]
    f = hparameters["f"]
    
    # Retrieve dimensions from A_prev's shape and dA's shape (≈2 lines)
    m, n_H_prev, n_W_prev, n_C_prev = A_prev.shape
    m, n_H, n_W, n_C = dA.shape
    
    # Initialize dA_prev with zeros (≈1 line)
    dA_prev = np.zeros(A_prev.shape)
    
    for i in range(m):                       # loop over the training examples
        
        # select training example from A_prev (≈1 line)
        a_prev = A_prev[i]
        
        for h in range(n_H):                   # loop on the vertical axis
            for w in range(n_W):               # loop on the horizontal axis
                for c in range(n_C):           # loop over the channels (depth)
                    
                    # Find the corners of the current "slice" (≈4 lines)
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    
                    # Compute the backward propagation in both modes.
                    if mode == "max":
                        
                        # Use the corners and "c" to define the current slice from a_prev (≈1 line)
                        a_prev_slice = a_prev[vert_start: vert_end, horiz_start: horiz_end, c]
                        # Create the mask from a_prev_slice (≈1 line)
                        mask = create_mask_from_window(a_prev_slice)
                        # Set dA_prev to be dA_prev + (the mask multiplied by the correct entry of dA) (≈1 line)
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] += mask * dA[i, h, w, c]
                        
                    elif mode == "average":
                        
                        # Get the value a from dA (≈1 line)
                        da = dA[i, h, w, c]
                        # Define the shape of the filter as fxf (≈1 line)
                        shape = (f,f)
                        # Distribute it to get the correct slice of dA_prev. i.e. Add the distributed value of da. (≈1 line)
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] += distribute_value(da, shape)
                        
    ### END CODE ###
    
    # Making sure your output shape is correct
    assert(dA_prev.shape == A_prev.shape)
    
    return dA_prev

In [ ]:
train_x_orig, train_y, test_x_orig, test_y = load_data_real()

In [ ]:
index = 15000
plt.imshow(train_x_orig[index])
if train_y[0,index] == 1:
    print ("y = " + str(train_y[0,index]) + ". Es la imagen de un gato.")
else:
    print ("y = " + str(train_y[0,index]) + ". NO es la imagen de un gato.")

In [ ]:
m_train = train_x_orig.shape[0]
num_px = train_x_orig.shape[1]
m_test = test_x_orig.shape[0]

print ("Number of training examples: " + str(m_train))
print ("Number of testing examples: " + str(m_test))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_x_orig shape: " + str(train_x_orig.shape))
print ("train_y shape: " + str(train_y.shape))
print ("test_x_orig shape: " + str(test_x_orig.shape))
print ("test_y shape: " + str(test_y.shape))

In [ ]:

def fit(X, y, epochs = 4, learning_rate=0.0075):
    
    np.random.seed(1)
    a_slice_prev = np.random.randn(4, 4, 3)
    W1 = np.random.randn(7, 7, 3, 10)
    b1 = np.random.randn(1, 1, 1, 10)
    
    W2 = np.random.randn(7, 7, 10, 15)
    b2 = np.random.randn(1, 1, 1, 15)
    
    hparameters1 = {
        "f": 4,
        "stride": 1,
        "pad": 1
    }
    hparameters2 = {
        "f":3,
        "stride": 2,
        "pad": 2
    }
    
    for i in range(epochs):
        print(f"Epoch {i} start")
        
        A1, cache1 = conv_forward(X, W1, b1, hparameters1)
        A_pool_1, cache_pool_1 = pool_forward(A1, hparameters1, mode = "max")
        print(f"Epoch {i} 20%")
        A2, cache2 = con_forward(A_pool_1, W2, b2, hparameters2)
        A_pool_2, cache_pool_2 = pool_forward(A2, hparameters2, mode = "average")
        print(f"Epoch {i} 40%")
        dA_2 = pool_backward(A_pool_2, cache_pool_2, mode = "average")
        dA_prev_2, dW2, db2 = conv_backward(dA_2, cache2)
        print(f"Epoch {i} 60%")
        
        dA_1 = pool_backward(dA_prev_2, cache_pool_1, mode = "max")
        dA_prev1, dW1, db1 = conv_backward(dA_1, cache1)
        print(f"Epoch {i} 85%")
        W1 -= learning_rate * dW1
        b1 -= learning_rate * db1
        
        W2 -= learning_rate * dW2
        b2 -= learning_rate * db2
        print(f"Epoch {i} 100%")
        
    # Redimensionamos los conjuntos
    train_x_flatten = A_pool_2.reshape(A_pool_2.shape[0], -1, -1).T   # El "-1" indica que se desea aplanar la curva! 
    print(train_x_flatten.shape)
    # Estandarizamos los valores de las features entre 0 y 1.
    train_x = train_x_flatten
    
    print("train_x's shape: " + str(train_x.shape))
    print("test_x's shape: " + str(test_x.shape))
    
    layers_dims = [20, 9] #  4-layer model
    deep_neural_network = DeepNeuralNetwork()
    deep_neural_network.fit(train_x, y, learning_rate = 0.0075, num_iterations = 2500, layer_hidden_neurons = layers_dims, activation="relu")
    
    return deep_neural_network
        
        
deep_neural_network = fit(train_x_orig, train_y)